<a href="https://colab.research.google.com/github/AkashR04/CUAD-summarization/blob/main/Summarization_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contract summarization for cuad dataset

## Dependencies

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## BART summarization


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model = "ainize/bart-base-cnn")
summary = summarizer(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

[{'summary_text': "Stremick's Heritage Foods, LLC and Premier Nutrition Corporation have agreed to amend their Manufacturing Agreement .\nThe Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following .\nAll other terms, conditions and covenants of the Agreement shall remain in full force and effect ."}]


## Pegasus summarization

In [ ]:
from transformers import pipeline
summarizer2 = pipeline("summarization", model = "google/pegasus-billsum")
summary = summarizer2(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

[{'summary_text': '(This measure has not been amended since it was reported to the House on July 9, 2018. (Sec. 1) This bill amends the Manufacturing Agreement between Heritage Foods, LLC, and Premier Nutrition Corporation (the parties to the manufacturing agreement) to revise the terms and conditions of the agreement.'}]


## T5 Summarization

In [ ]:
summarizer3 = pipeline("summarization", model = "stevhliu/t5-small-finetuned-billsum-ca_test")
summary = summarizer3(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'summary_text': 'AMENDMENT NO. 1 TO STREMICK\'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT This Amendment is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). The amendment would require the Parties to remove Section 2(c) in its entirety and replace it with the following changes shall be made to the Agreement. This amendment would authorize the Partie to amend the Agreement in accordance with the terms and conditions set forth herein. This Amendment, signed by and between Stremick\'s Heritage Foods, LLC ("Heritage") is effective on June 11, 2018, and amend that certain manufacturing Agreement between and between Premier Nutrition Corporation ("Premier"'}]
